In [1]:
import socket
import threading

In [2]:
def receive_messages(client_socket):
    while True:
        try:
            message = client_socket.recv(1024).decode()
            print(message)
        except:
            print("An error occurred!")
            client_socket.close()
            break

In [3]:
def main():
    host = '127.0.0.1'
    port = 12347

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen()

    print("Chat server started on {}:{}".format(host, port))

    clients = []

    while True:
        client_socket, addr = server_socket.accept()
        print("Connected with", addr)
        clients.append(client_socket)
        threading.Thread(target=receive_messages, args=(client_socket,)).start()

        # Sending welcome message to newly connected client
        welcome_msg = "Welcome to the chat server! Type 'exit' to quit."
        client_socket.send(welcome_msg.encode())

        # Broadcasting to all clients that a new client has joined
        broadcast_msg = "New user joined the chat: {}".format(addr)
        for client in clients:
            if client != client_socket:
                client.send(broadcast_msg.encode())

        # Receiving messages from clients
        while True:
            try:
                message = client_socket.recv(1024).decode()
                if message.lower() == 'exit':
                    client_socket.close()
                    clients.remove(client_socket)
                    print("Connection closed by", addr)
                    # Broadcasting to all clients that a client has left
                    left_msg = "User {} left the chat.".format(addr)
                    for client in clients:
                        if client != client_socket:
                            client.send(left_msg.encode())
                    break
                else:
                    print(addr, ":", message)
                    # Broadcasting received message to all clients except the sender
                    broadcast_msg = "{} : {}".format(addr, message)
                    for client in clients:
                        if client != client_socket:
                            client.send(broadcast_msg.encode())
            except:
                break

    server_socket.close()

In [ ]:
if __name__ == "__main__":
    main()

Chat server started on 127.0.0.1:12347
Connected with ('127.0.0.1', 63042)
Hello world!
('127.0.0.1', 63042) : Welcome
